In [2]:
import pandas as pd

In [3]:
# #tylko w colab
# from google.colab import drive
# drive.mount('/content/drive')

# import os
# os.listdir('/content/drive/MyDrive')

# df = pd.read_csv('/content/drive/MyDrive/Zestawienie operacji za 01.01.2010 - 28.11.2025.csv', encoding="latin2")

# df.head()

# from google.colab import files
# import pandas as pd

# countries_df.to_excel("countries_df.xlsx", index=True)
# files.download("countries_df.xlsx")


In [ ]:
file_name = "data/account statements/Zestawienie operacji za 01.01.2010 - 28.11.2025.csv" 
df = pd.read_csv(file_name, encoding="latin2")



,Data operacji,Data waluty,Typ transakcji,Kwota,Waluta,Opis transakcji,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,2025-11-28,2025-11-26,Płatnoć kartš,-31.00,PLN,Tytuł: 010061097 74169505330633307706340,Lokalizacja: Adres: OLLI S C Miasto: WARSZAWA ...,Data wykonania operacji: 2025-11-26,Oryginalna kwota operacji: 31.00,Numer karty: 425125******8930,NaN,NaN,NaN,NaN
1,2025-11-28,2025-11-26,Płatnoć kartš,-72.16,PLN,Tytuł: 010059127 74043215330148157931508,Lokalizacja: Adres: Apteka przy Domaniewskiej ...,Data wykonania operacji: 2025-11-26,Oryginalna kwota operacji: 72.16,Numer karty: 425125******8930,NaN,NaN,NaN,NaN
2,2025-11-28,2025-11-26,Płatnoć kartš,-50.22,PLN,Tytuł: 74988855330496415991685,Lokalizacja: Adres: JMP S.A. BIEDRONKA 7224 Mi...,Data wykonania operacji: 2025-11-26,Oryginalna kwota operacji: 50.22,Numer karty: 425125******8930,NaN,NaN,NaN,NaN
3,2025-11-27,2025-11-27,Przelew na konto,6281.91,PLN,Rachunek nadawcy :,06 1240 0030 1111 1111 1111 5990,Nazwa nadawcy: Bank PEKAO S.A. Żubra 1,Adres nadawcy: 01-066 Warszawa,Tytuł: Wynagrodzenie,NaN,NaN,NaN,NaN
4,2025-11-25,2025-11-23,Płatnoć kartš,-4.30,PLN,Tytuł: 000498849 74230785327180203293377,Lokalizacja: Adres: ZABKA Z5795 K.1 Miasto: SI...,Data wykonania operacji: 2025-11-23,Oryginalna kwota operacji: 4.30,Numer karty: 425125******8930,NaN,NaN,NaN,NaN


In [6]:
# countries = (
#     df['Unnamed: 6']
#     .str.extract(r"Kraj:\s*(.*)", expand=False)
#     .str.strip()
# )

# countries_df=countries.value_counts()
# countries_df


In [7]:
#creating schema of output, cleaned df

df_std = pd.DataFrame(index=df.index)

df_std["transaction_id"] = None
df_std["date"] = df["Data waluty"]
df_std["transaction_type"] = df["Typ transakcji"]
df_std["amount"] = (
    df["Kwota"]
    .astype(str)
    .str.replace(" ", "", regex=False)
    .str.replace(",", ".", regex=False)
    .astype(float)
)
df_std["currency"] = df["Waluta"]
df_std["description"] = None
df_std["merchant"] = None
df_std["country"] = None
df_std["city"] = None
df_std["category"] = None
df_std["subcategory"] = None
df_std["rule_id"] = None


In [8]:
#dictionary to map transaction type names - translate and standarize
transaction_type_map = {
    "Płatność kartą": "card_payment",
    "Płatnoć kartš": "card_payment",

    "Zakup w terminalu - kod mobilny": "terminal_payment_blik",

    "Autooszczędzanie": "auto_savings",

    "Płatność web - kod mobilny": "web_payment_blik",
    "Płatnoć web - kod mobilny": "web_payment_blik",

    "Przelew na telefon przychodz. zew.": "transfer_blik",
    "Przelew na telefon przychodz. wew.": "transfer_blik",

    "Przelew na konto": "transfer_to_account",
    "Przelew z rachunku": "transfer_from_account",

    "Opłata za użytkowanie karty": "card_fee",

    "Zwrot w terminalu": "refund",
    "Zwrot płatności kartą": "refund",
    "Zwrot płatnoci kartš": "refund",

    "Zlecenie stałe": "standing_order",

    "Wypłata z bankomatu": "atm_withdrawal",
    "Wypłata w bankomacie - kod mobilny": "atm_withdrawal",
    "Wypłata w bankomacie - czek": "atm_withdrawal",

    "Obciążenie": "pko_charge",
    "Obcišżenie": "pko_charge",

    "Uznanie": "pko_credit",

    "Naliczenie odsetek": "interest_accrued"
}

In [9]:
#function to map
def normalize_transaction_type(value, mapping):
    if pd.isna(value):
        return "unknown"
    cleaned = str(value).strip()
    if cleaned in mapping:
        return mapping[cleaned]

    # if not in the mapping return cleaned"
    return cleaned


In [10]:
df_std["transaction_type"] = df["Typ transakcji"].apply(
    lambda x: normalize_transaction_type(x, transaction_type_map)
)

In [11]:
def find_column_with_prefix(row, prefix):
    """
    Search the row for any column whose text starts with the given prefix.
    Returns the matching cell value as a string, or an empty string if not found.
    """
    row_str = row.astype(str)

    # Check which columns start with the given prefix
    matches = row_str.str.startswith(prefix, na=False)

    # If no column starts with this prefix, return empty string
    if not matches.any():
        return ""

    # Return the first matched column's value
    return row_str[matches].iloc[0]

processing of first two transaction types payments in terminal - by card and blik


In [12]:
#mask on transaction type (we have same index for df and df_std)
mask = df_std["transaction_type"].isin(["card_payment", "terminal_payment_blik"])


# extraction of transaction id
df_std.loc[mask, "transaction_id"] = (
    df.loc[mask]
      .apply(
          lambda row: pd.Series([
              find_column_with_prefix(
                  row,
                  "Tytuł:" 
                  if df_std.loc[row.name, "transaction_type"] == "card_payment"
                  else "Numer referencyjny:"
              )
          ])
          .str.extract(
              r"Tytuł:\s*([\d\s]+)" 
              if df_std.loc[row.name, "transaction_type"] == "card_payment"
              else r"Numer referencyjny:\s*([\d\s]+)",
              expand=False
          )
          .fillna("")
          .str.replace(" ", "", regex=False)
          .str.strip()
          .iloc[0],
          axis=1
      )
)

  

# extraction of the raw DESCRIPTION
df_std.loc[mask, "description"] = (
    df.loc[mask]
      .apply(lambda row: find_column_with_prefix(row, "Lokalizacja:"), axis=1)
      .str.extract(r"Adres:\s*(.*?)\s*Miasto:", expand=False)
      .str.strip()
      .str.upper()
)


# extracion of the CITY"
df_std.loc[mask, "city"] = (
    df.loc[mask]
      .apply(lambda row: find_column_with_prefix(row, "Lokalizacja:"), axis=1)
      .str.extract(r"Miasto:\s*([A-Za-zĄĆĘŁŃÓŚŹŻąćęłńóśźż0-9\s]+?)\s*Kraj:", expand=False)
      .str.strip()
      .str.upper()
)

# extraction of the COUNTRY
df_std.loc[mask, "country"] = (
    df.loc[mask]
      .apply(lambda row: find_column_with_prefix(row, "Lokalizacja:"), axis=1)
      .str.extract(r"Kraj:\s*(.*)", expand=False)
      .str.strip()
      .str.upper()
)



processing the second transaction type - web payment (basically blik web payment)

In [13]:
#mask on transaction type (we have same index for df and df_std)
mask = df_std["transaction_type"].isin(["web_payment_blik", "refund"])

# extraction of transaction id
df_std.loc[mask, "transaction_id"] = (
    df.loc[mask]
      .apply(lambda row: find_column_with_prefix(row, "Numer referencyjny:"), axis=1)
      .fillna("")
      .str.extract(r"Numer referencyjny:\s*([\d\s]+)", expand=False)
      .fillna("")
      .str.replace(" ", "", regex=False)
      .str.strip()
)

# extraction of the raw DESCRIPTION
df_std.loc[mask, "description"] = (
    df.loc[mask]
      .apply(lambda row: find_column_with_prefix(row, "Lokalizacja:"), axis=1)
      .str.extract(r"Adres:\s*(.*)", expand=False)
      .str.strip()
      .str.upper()
)

processing of transfers

In [14]:
import hashlib

def short_hash(*values, length=4):
    base = "|".join(str(v) for v in values)
    h = hashlib.sha256(base.encode()).hexdigest()
    return h[:length]

In [15]:
#mask on transaction type (we have same index for df and df_std)
mask = df_std["transaction_type"].isin(["transfer_blik", "standing_order", "transfer_to_account", "transfer_from_account"])

# extract Tytuł → until OD: or end of string
df_std.loc[mask, "description"] = (
    df.loc[mask]
      .apply(lambda row: find_column_with_prefix(row, "Tytuł:"), axis=1)
      .str.extract(r"Tytuł:\s*(.*?)(?:\s*OD:|$)", expand=False)
      .fillna("")
      .str.strip()
      .str.upper()
      .apply(lambda x: (
          x if x == "PRZELEW NA TELEFON" 
          else x.replace("PRZELEW NA TELEFON", "").strip()
      ))
)

# extraction of transaction id
account_numbers = (
    df["Unnamed: 6"]
    .astype(str)
    .str.replace(" ", "", regex=False)
    .str.strip()
)

df_std.loc[mask, "transaction_id"] = df_std.loc[mask].apply(
    lambda row: (
        account_numbers[row.name] + " " + 
        short_hash(row["date"], row["amount"], row["description"], length=4)
    ),
    axis=1
)

processing of atm withdrawal

In [16]:
mask = df_std["transaction_type"].isin(["atm_withdrawal"])

# extraction of the raw DESCRIPTION
# Step 1: choose the correct source text per row
source_text = df.loc[mask].apply(
    lambda row: (
        find_column_with_prefix(row, "Numer referencyjny:")
        if find_column_with_prefix(row, "Numer referencyjny:") != ""
        else find_column_with_prefix(row, "Tytuł:")
    ),
    axis=1
)

# Step 2: extract ID from either format
df_std.loc[mask, "transaction_id"] = (
    source_text
      .fillna("")
      .str.extract(r"(?:Numer referencyjny|Tytuł):\s*([\d\s]+)", expand=False)
      .fillna("")
      .str.replace(" ", "", regex=False)
      .str.strip()
)

# extraction of the raw DESCRIPTION
df_std.loc[mask, "description"] = (
    df.loc[mask]
      .apply(lambda row: find_column_with_prefix(row, "Lokalizacja:"), axis=1)
      .str.extract(r"Adres:\s*(.*?)\s*Miasto:", expand=False)
      .str.strip()
      .str.upper()
)


# extracion of the CITY"
df_std.loc[mask, "city"] = (
    df.loc[mask]
      .apply(lambda row: find_column_with_prefix(row, "Lokalizacja:"), axis=1)
      .str.extract(r"Miasto:\s*([A-Za-zĄĆĘŁŃÓŚŹŻąćęłńóśźż0-9\s]+?)\s*Kraj:", expand=False)
      .str.strip()
      .str.upper()
)

# extraction of the COUNTRY
df_std.loc[mask, "country"] = (
    df.loc[mask]
      .apply(lambda row: find_column_with_prefix(row, "Lokalizacja:"), axis=1)
      .str.extract(r"Kraj:\s*(.*)", expand=False)
      .str.strip()
      .str.upper()
)


processing some different low key types of transactions

In [17]:
#autosavings
mask = df_std["transaction_type"].isin(["auto_savings"])

df["Kwota_next"] = df["Kwota"].shift(-1)


# extraction (or more like creation) of transaction id
df_std.loc[mask, "transaction_id"] = df.loc[mask].apply(
    lambda row: short_hash(
        row["Data operacji"],
        row["Data waluty"],
        row["Typ transakcji"],
        row["Kwota"],
        row["Kwota_next"],  # amount from a line below
        length=6
    ),
    axis=1
)

# setting description
df_std.loc[mask, "description"] = "auto_savings"

In [19]:
import re
from collections import Counter

# 1. Weź kolumnę description
descriptions = df_std["description"].dropna().astype(str)

# 2. Połącz wszystko w jeden wielki tekst
all_text = " ".join(descriptions)

# 3. Wyczyść z syfów i znormalizuj
all_text = all_text.upper()
all_text = re.sub(r"[^A-Z0-9 ]", " ", all_text)   # zostaw tylko litery/cyfry/spacje
all_text = re.sub(r"\s+", " ", all_text).strip()  # kompresja spacji

# 4. Podziel na tokeny
tokens = all_text.split(" ")

# 5. Policz częstotliwość
counts = Counter(tokens)

# 6. Usuń tokeny nic-nie-mówiące, jeśli chcesz
stopwords = {"PL", "UL", "HTTP", "HTTPS", "WWW", ""}

for s in stopwords:
    counts.pop(s, None)

# 7. Pokaż Top 50
counts.most_common(50)


[('AUTO', 1417),
 ('SAVINGS', 1417),
 ('K', 782),
 ('ZABKA', 704),
 ('S', 492),
 ('1', 361),
 ('2', 341),
 ('A', 322),
 ('JMP', 255),
 ('BIEDRONKA', 255),
 ('Z6860', 240),
 ('EXPRESS', 232),
 ('CARREFOUR', 228),
 ('PIEKARNIE', 166),
 ('LUBASZKA', 166),
 ('WARSZAWA', 148),
 ('AUTOMATY', 146),
 ('24', 138),
 ('ZC015', 129),
 ('MCDONALDS', 127),
 ('F95', 117),
 ('NA', 115),
 ('PRZELEW', 114),
 ('CYGARO', 108),
 ('EWA', 108),
 ('KAFAR', 108),
 ('TELEFON', 102),
 ('KFC', 98),
 ('UVP', 90),
 ('ALLEGRO', 88),
 ('GRUBA', 72),
 ('OLX', 72),
 ('747', 71),
 ('W', 61),
 ('CONCEPT', 54),
 ('MM', 53),
 ('SP', 51),
 ('Z', 50),
 ('O', 49),
 ('COM', 45),
 ('BAR', 44),
 ('GOOGLE', 42),
 ('48696', 41),
 ('013', 41),
 ('WA', 39),
 ('3', 38),
 ('STYPENDIUM', 37),
 ('01', 36),
 ('DELIKATESY', 36),
 ('UW', 36)]